In [40]:
from __future__ import absolute_import
import json
import logging
import os
import time
from argparse import ArgumentParser
from datetime import datetime

from pgportfolio.tools.configprocess import preprocess_config
from pgportfolio.tools.configprocess import load_config
from pgportfolio.tools.trade import save_test_data
from pgportfolio.tools.shortcut import execute_backtest
from pgportfolio.resultprocess import plot


def build_parser():
    parser = ArgumentParser()
    parser.add_argument("--mode",dest="mode",
                        help="start mode, train, generate, download_data"
                             " backtest",
                        metavar="MODE", default="train")
    parser.add_argument("--processes", dest="processes",
                        help="number of processes you want to start to train the network",
                        default="1")
    parser.add_argument("--repeat", dest="repeat",
                        help="repeat times of generating training subfolder",
                        default="1")
    parser.add_argument("--algo",
                        help="algo name or indexes of training_package ",
                        dest="algo")
    parser.add_argument("--algos",
                        help="algo names or indexes of training_package, seperated by \",\"",
                        dest="algos")
    parser.add_argument("--labels", dest="labels",
                        help="names that will shown in the figure caption or table header")
    parser.add_argument("--format", dest="format", default="raw",
                        help="format of the table printed")
    parser.add_argument("--device", dest="device", default="cpu",
                        help="device to be used to train")
    parser.add_argument("--folder", dest="folder", type=int,
                        help="folder(int) to load the config, neglect this option if loading from ./pgportfolio/net_config")
    return parser


def main():
    parser = build_parser()
    options = parser.parse_args()
    if not os.path.exists("./" + "train_package"):
        os.makedirs("./" + "train_package")
    if not os.path.exists("./" + "database"):
        os.makedirs("./" + "database")

    if options.mode == "train":
        import pgportfolio.autotrain.training
        if not options.algo:
            pgportfolio.autotrain.training.train_all(int(options.processes), options.device)
        else:
            for folder in options.folder:
                raise NotImplementedError()
    elif options.mode == "generate":
        import pgportfolio.autotrain.generate as generate
        logging.basicConfig(level=logging.INFO)
        generate.add_packages(load_config(), int(options.repeat))
    elif options.mode == "download_data":
        from pgportfolio.marketdata.datamatrices import DataMatrices
        with open("./pgportfolio/net_config.json") as file:
            config = json.load(file)
        config = preprocess_config(config)
        start = time.mktime(datetime.strptime(config["input"]["start_date"], "%Y/%m/%d").timetuple())
        end = time.mktime(datetime.strptime(config["input"]["end_date"], "%Y/%m/%d").timetuple())
        DataMatrices(start=start,
                     end=end,
                     feature_number=config["input"]["feature_number"],
                     window_size=config["input"]["window_size"],
                     online=True,
                     period=config["input"]["global_period"],
                     volume_average_days=config["input"]["volume_average_days"],
                     coin_filter=config["input"]["coin_number"],
                     is_permed=config["input"]["is_permed"],
                     test_portion=config["input"]["test_portion"],
                     portion_reversed=config["input"]["portion_reversed"])
    elif options.mode == "backtest":
        config = _config_by_algo(options.algo)
        _set_logging_by_algo(logging.DEBUG, logging.DEBUG, options.algo, "backtestlog")
        execute_backtest(options.algo, config)
    elif options.mode == "save_test_data":
        # This is used to export the test data
        save_test_data(load_config(options.folder))
    elif options.mode == "plot":
        logging.basicConfig(level=logging.INFO)
        algos = options.algos.split(",")
        if options.labels:
            labels = options.labels.replace("_"," ")
            labels = labels.split(",")
        else:
            labels = algos
        plot.plot_backtest(load_config(), algos, labels)
    elif options.mode == "table":
        algos = options.algos.split(",")
        if options.labels:
            labels = options.labels.replace("_"," ")
            labels = labels.split(",")
        else:
            labels = algos
        plot.table_backtest(load_config(), algos, labels, format=options.format)

def _set_logging_by_algo(console_level, file_level, algo, name):
    if algo.isdigit():
            logging.basicConfig(filename="./train_package/"+algo+"/"+name,
                                level=file_level)
            console = logging.StreamHandler()
            console.setLevel(console_level)
            logging.getLogger().addHandler(console)
    else:
        logging.basicConfig(level=console_level)


def _config_by_algo(algo):
    """
    :param algo: a string represent index or algo name
    :return : a config dictionary
    """
    if not algo:
        raise ValueError("please input a specific algo")
    elif algo.isdigit():
        config = load_config(algo)
    else:
        config = load_config()
    return config



In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
from pgportfolio.marketdata.datamatrices import DataMatrices

In [17]:
with open("./pgportfolio/net_config.json") as file:
    config = json.load(file)
config = preprocess_config(config)

start = time.mktime(datetime.strptime(config["input"]["start_date"], "%Y/%m/%d").timetuple())
end = time.mktime(datetime.strptime(config["input"]["end_date"], "%Y/%m/%d").timetuple())

In [8]:
a=DataMatrices(start=start,
                     end=end,
                     feature_number=config["input"]["feature_number"],
                     window_size=config["input"]["window_size"],
                     online=True,
                     period=config["input"]["global_period"],
                     volume_average_days=config["input"]["volume_average_days"],
                     coin_filter=config["input"]["coin_number"],
                     is_permed=config["input"]["is_permed"],
                     test_portion=config["input"]["test_portion"],
                     portion_reversed=config["input"]["portion_reversed"])

                   pair         volume     price
coin                                            
ETH             BTC_ETH  833820.402758  0.020135
LTC             BTC_LTC  669757.028884  0.006100
XRP             BTC_XRP  465788.871570  0.000031
reversed_USDT  USDT_BTC  200241.369072  0.000138
ETC             BTC_ETC  165836.536867  0.000522
DASH           BTC_DASH  145825.768286  0.006920
XMR             BTC_XMR   82908.949198  0.007287
XEM             BTC_XEM   69047.989435  0.000005
FCT             BTC_FCT   63216.478905  0.000389
GNT             BTC_GNT   59158.689084  0.000005
ZEC             BTC_ZEC   51503.928711  0.004207


/Users/lijinyang/Documents/GitHub/PGPortfolio/pgportfolio/marketdata/datamatrices.py:51: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  features=type_list)
/Users/lijinyang/Documents/GitHub/PGPortfolio/pgportfolio/marketdata/globaldatamatrix.py:119: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  panel = panel_fillna(panel, "both")


https://www.google.com/finance/getprices?i=60&p=10d&f=d,o,h,l,c,v&df=cpct&q=GOOG


In [18]:
(end-start)*0.92/1800

32280.96

In [20]:
from pgportfolio.tools.data import get_volume_forward, get_type_list

In [21]:
volume_forward = get_volume_forward(end-start, 0.08, False)

In [22]:
volume_forward

5052672.0

In [84]:
import pgportfolio.marketdata.globaldatamatrix as gdm
t=gdm.HistoryManager(coin_number=11, end=end, volume_average_days=30,
                                                    volume_forward=volume_forward, online=True)

In [38]:
from pgportfolio.constants import *

In [42]:
from pgportfolio.marketdata.coinlist import CoinList
c=CoinList(end, 30, volume_forward)

In [65]:
c.topNVolume(11).loc['ETC']['price']

                   pair         volume     price
coin                                            
ETH             BTC_ETH  833820.402758  0.020145
LTC             BTC_LTC  669757.028884  0.006082
XRP             BTC_XRP  465788.871570  0.000031
reversed_USDT  USDT_BTC  200241.369072  0.000138
ETC             BTC_ETC  165836.536867  0.000523
DASH           BTC_DASH  145825.768286  0.006960
XMR             BTC_XMR   82908.949198  0.007304
XEM             BTC_XEM   69047.989435  0.000005
FCT             BTC_FCT   63216.478905  0.000386
GNT             BTC_GNT   59158.689084  0.000005
ZEC             BTC_ZEC   51503.928711  0.004199


0.00052287

In [47]:
DAY

86400

In [48]:
24*60*60

86400

In [49]:
end

1498881600.0

In [50]:
DAY*30

2592000

In [9]:
import pandas as pd

In [11]:
s=pd.read_csv('./Raw Data/ETH_20171211.csv')

In [48]:
mm=datetime.utcfromtimestamp(1469334000).strftime('%Y-%m-%d %H:%M:%S')

In [68]:
pd.to_datetime(1513053300, unit='s', origin='unix') 

Timestamp('2017-12-12 04:35:00')

In [50]:
pd.to_datetime(mm)

Timestamp('2016-07-24 04:20:00')

In [43]:
pd.to_datetime(mm).astype(int) / 10**9

AttributeError: 'Timestamp' object has no attribute 'astype'

In [52]:
pd.to_datetime([mm]).astype(int) / 10**9

Float64Index([1469334000.0], dtype='float64')

In [56]:
time.localtime(1469334000)

time.struct_time(tm_year=2016, tm_mon=7, tm_mday=24, tm_hour=0, tm_min=20, tm_sec=0, tm_wday=6, tm_yday=206, tm_isdst=1)

In [13]:
s['date_readable'] = pd.to_datetime(s['date'],unit='s')

In [14]:
s.to_csv('./Raw Data/ETH_readable_20171211.csv')

In [66]:
from pgportfolio.marketdata.poloniex import Poloniex
polo = Poloniex()
# connect the internet to accees volumes
vol = polo.marketVolume()
ticker = polo.marketTicker()
pairs = []
coins = []
volumes = []
prices = []

In [69]:
volume_forward/1800

2807.04

In [24]:
d=b.get_global_panel(start,end,1800,['close', 'high', 'low'])

                   pair         volume     price
coin                                            
ETH             BTC_ETH  833820.402758  0.019858
LTC             BTC_LTC  669757.028884  0.006045
XRP             BTC_XRP  465788.871570  0.000031
reversed_USDT  USDT_BTC  200241.369072  0.000139
ETC             BTC_ETC  165836.536867  0.000524
DASH           BTC_DASH  145825.768286  0.006879
XMR             BTC_XMR   82908.949198  0.007365
XEM             BTC_XEM   69047.989435  0.000005
FCT             BTC_FCT   63216.478905  0.000385
GNT             BTC_GNT   59158.689084  0.000005
ZEC             BTC_ZEC   51503.928711  0.004440


In [126]:
config["input"]["end_date"]

'2017/07/01'

In [113]:
d.major_axis

Index(['ETH', 'LTC', 'XRP', 'reversed_USDT', 'ETC', 'DASH', 'XMR', 'XEM',
       'FCT', 'GNT', 'ZEC'],
      dtype='object')

In [79]:
d.minor_axis

DatetimeIndex(['2015-07-01 04:00:00', '2015-07-01 04:30:00',
               '2015-07-01 05:00:00', '2015-07-01 05:30:00',
               '2015-07-01 06:00:00', '2015-07-01 06:30:00',
               '2015-07-01 07:00:00', '2015-07-01 07:30:00',
               '2015-07-01 08:00:00', '2015-07-01 08:30:00',
               ...
               '2017-06-30 23:30:00', '2017-07-01 00:00:00',
               '2017-07-01 00:30:00', '2017-07-01 01:00:00',
               '2017-07-01 01:30:00', '2017-07-01 02:00:00',
               '2017-07-01 02:30:00', '2017-07-01 03:00:00',
               '2017-07-01 03:30:00', '2017-07-01 04:00:00'],
              dtype='datetime64[ns]', length=35089, freq=None)

In [30]:
PVM = pd.DataFrame(index=d.minor_axis,
                          columns=d.major_axis)
PVM = PVM.fillna(1.0 / 11)

window_size = 50
num_periods = len(d.minor_axis)

In [31]:
PVM

,ETH,LTC,XRP,reversed_USDT,ETC,DASH,XMR,XEM,FCT,GNT,ZEC
2015-07-01 04:00:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 04:30:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 05:00:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 05:30:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 06:00:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 06:30:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 07:00:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 07:30:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 08:00:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
2015-07-01 08:30:00,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909


In [48]:
PVM.iloc[25285]

ETH              0.090909
LTC              0.090909
XRP              0.090909
reversed_USDT    0.090909
ETC              0.090909
DASH             0.090909
XMR              0.090909
XEM              0.090909
FCT              0.090909
GNT              0.090909
ZEC              0.090909
Name: 2016-12-08 22:30:00, dtype: float64

In [33]:
32249+2776

35025

In [1]:
from pgportfolio.marketdata.datamatrices import DataMatrices

In [2]:
from pgportfolio.tools.configprocess import load_config

In [41]:
q=DataMatrices.create_from_config(load_config(1))

In [127]:
batch,b = q.next_batch()
batch_input = batch["X"]
batch_y = batch["y"]
batch_last_w = batch["last_w"]
batch_w = batch["setw"]

In [79]:
import numpy as np
b = np.array(b)
PVM.values[b-1, :]

array([[0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909],
       [0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909],
       [0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909],
       ...,
       [0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909],
       [0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909],
       [0.09090909, 0.09090909, 0.09090909, ..., 0.09090909, 0.09090909,
        0.09090909]])

In [128]:
min(b),max(b),len(b)

(29811, 29919, 109)

In [88]:
d

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 11 (major_axis) x 35089 (minor_axis)
Items axis: close to low
Major_axis axis: ETH to ZEC
Minor_axis axis: 2015-07-01 04:00:00 to 2017-07-01 04:00:00

In [104]:
batch_input.shape,batch_y.shape

((109, 3, 11, 31), (109, 3, 11))

In [107]:
batch_last_w.shape

(109, 11)

In [109]:
batch_w

<function pgportfolio.marketdata.datamatrices.DataMatrices.__pack_samples.<locals>.setw(w)>

In [81]:
batch_input[0,0,0,:]

array([0.03989976, 0.03943419, 0.0394    , 0.03993396, 0.041055  ,
       0.04051   , 0.04032008, 0.04022   , 0.04061   , 0.040712  ,
       0.04054992, 0.04097   , 0.040565  , 0.04021001, 0.04009   ,
       0.04033   , 0.04024399, 0.04030103, 0.040306  , 0.04054401,
       0.04076423, 0.04187798, 0.04258751, 0.042753  , 0.04250966,
       0.04345   , 0.0437    , 0.0431    , 0.04251012, 0.04289988,
       0.04230884], dtype=float32)

In [115]:
batch_input[1,0,0,:]

array([0.01204   , 0.01205766, 0.012047  , 0.01206881, 0.01199953,
       0.01200101, 0.01203998, 0.012037  , 0.01204847, 0.01204558,
       0.01202712, 0.012023  , 0.0117808 , 0.01130003, 0.01129005,
       0.01149781, 0.01160759, 0.011645  , 0.01161774, 0.011601  ,
       0.01155598, 0.011556  , 0.01156817, 0.01154802, 0.01154499,
       0.01153105, 0.0114    , 0.01148056, 0.01151   , 0.01155512,
       0.011532  ], dtype=float32)

In [58]:
s=pd.read_csv('./Raw Data/ETC_readable.csv')

In [60]:
s[s['date_readable']=="2016-12-08 22:30:00"]

,Unnamed: 0,date,coin,high,low,open,close,volume,quoteVolume,weightedAverage,date_readable
39674,39674,1481236200,ETC,0.001062,0.001062,0.001062,0.001062,0.096132,90.505795,0.001062,2016-12-08 22:30:00


In [109]:
d.values.shape

(3, 11, 35089)

In [115]:
d.values[:,4,:]

array([[0.00201001, 0.00201001, 0.00201001, ..., 0.00708969, 0.00709411,
        0.00708878],
       [0.01      , 0.01      , 0.01      , ..., 0.00709774, 0.00710101,
        0.00714362],
       [0.00060001, 0.00060001, 0.00060001, ..., 0.0070311 , 0.00705021,
        0.00706091]], dtype=float32)

In [91]:
window_size=31
def get_submatrix(ind):
    return d.values[:, :, ind:ind+window_size+1]

In [129]:
M = [get_submatrix(index) for index in b]
M = np.array(M)
X = M[:, :, :, :-1]
y = M[:, :, :, -1] / M[:, 0, None, :, -2]

In [134]:
X.shape, d.values.shape, M.shape, y.shape

((109, 3, 11, 31), (3, 11, 35089), (109, 3, 11, 32), (109, 3, 11))

In [135]:
PVM.shape

(35089, 11)

In [132]:
X[60,:,4,0]

array([0.00138099, 0.00138849, 0.0013789 ], dtype=float32)

In [105]:
batch_input[0,0,0,:]

array([0.03016388, 0.03024982, 0.0303    , 0.03070685, 0.03099929,
       0.03105385, 0.03093586, 0.03140082, 0.03167166, 0.0317    ,
       0.0315    , 0.03129267, 0.03129282, 0.03174889, 0.03141516,
       0.03128629, 0.03155387, 0.03142776, 0.031552  , 0.03168965,
       0.03186994, 0.032     , 0.03190064, 0.03188401, 0.03192   ,
       0.03183613, 0.03197554, 0.03149946, 0.03165   , 0.03168   ,
       0.03174   ], dtype=float32)

In [113]:
35089-32249-2776

64

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

In [141]:
plt.show('./Result/crp_olmar_nnagent 20191211_1/crp_olmar_nnagent.eps')
plt.show()

In [143]:
S

,Unnamed: 0,date,coin,high,low,open,close,volume,quoteVolume,weightedAverage,date_readable
0,0,1469334000,ETC,0.010000,0.006000,0.009950,0.006000,0.399950,46.666667,0.008570,2016-07-24 04:20:00
1,1,1469334300,ETC,0.006000,0.000600,0.000600,0.002010,0.232879,87.178785,0.002671,2016-07-24 04:25:00
2,2,1469334600,ETC,0.002780,0.000500,0.002020,0.002000,6.116057,4451.805492,0.001374,2016-07-24 04:30:00
3,3,1469334900,ETC,0.001799,0.000101,0.001799,0.000700,6.481349,13909.924200,0.000466,2016-07-24 04:35:00
4,4,1469335200,ETC,0.001700,0.000750,0.000850,0.000750,8.321861,7712.990676,0.001079,2016-07-24 04:40:00
5,5,1469335500,ETC,0.005599,0.000750,0.001050,0.003500,14.034133,5583.484243,0.002514,2016-07-24 04:45:00
6,6,1469335800,ETC,0.004387,0.000810,0.003869,0.000935,40.823221,30931.081400,0.001320,2016-07-24 04:50:00
7,7,1469336100,ETC,0.001088,0.000750,0.000950,0.000919,18.094139,20365.337640,0.000888,2016-07-24 04:55:00
8,8,1469336400,ETC,0.000919,0.000500,0.000850,0.000510,34.940305,45057.638360,0.000775,2016-07-24 05:00:00
9,9,1469336700,ETC,0.000627,0.000420,0.000615,0.000450,12.843110,27488.530120,0.000467,2016-07-24 05:05:00


In [58]:
stock=pd.read_table("./Raw Data/S&P500 from 20171212/AAPL.txt",sep=',',header=None)
stock.columns=['Date1', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
stock['Date'] = pd.to_datetime(stock['Date1'] + ' ' + stock['Time'])
stock = stock.drop(['Date1','Time'], axis=1)

In [61]:
stock['Date']=stock['Date'].astype(int) / 10**9

In [66]:
stock['Date']=pd.to_numeric(stock['Date'], downcast='integer')

In [77]:
def get_stock_table(ticker):
    stock=pd.read_table("./Raw Data/S&P500 from 20171212/"+ticker+".txt",sep=',',header=None)
    stock.columns=['Date1', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    stock['Date'] = pd.to_datetime(stock['Date1'] + ' ' + stock['Time'])
    stock = stock.drop(['Date1','Time'], axis=1)
    stock['Date']=stock['Date'].astype(int) / 10**9
    stock['Date']=pd.to_numeric(stock['Date'], downcast='integer')
    stock['coin']=ticker
    return stock

In [98]:
stock=pd.read_table("./Raw Data/S&P500 from 20171212/"+'AAPL'+".txt",sep=',',header=None)
stock.columns=['Date1', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
stock['Date'] = pd.to_datetime(stock['Date1'] + ' ' + stock['Time'])
stock = stock.drop(['Date1','Time'], axis=1)

In [100]:
stock

,Open,High,Low,Close,Volume,Date
0,167.2000,167.2000,167.2000,167.2000,154,2017-12-12 04:35:00
1,167.3800,167.3800,167.3800,167.3800,103,2017-12-12 04:55:00
2,167.2400,167.2400,167.2400,167.2400,208,2017-12-12 05:05:00
3,167.2000,167.2000,167.2000,167.2000,280,2017-12-12 05:10:00
4,166.9800,167.2400,166.9800,167.2400,352,2017-12-12 05:35:00
5,167.2000,167.2000,167.2000,167.2000,515,2017-12-12 05:55:00
6,167.2600,167.2600,167.2600,167.2600,109,2017-12-12 06:30:00
7,167.2900,167.2900,167.2600,167.2600,1148,2017-12-12 06:50:00
8,167.3300,167.3300,167.3300,167.3300,103,2017-12-12 07:00:00
9,167.3800,167.3800,167.3800,167.3800,206,2017-12-12 07:05:00


In [1]:
from sqlalchemy import create_engine

In [11]:
import sqlite3

In [94]:
conn = sqlite3.connect('TestDB1.db')
c = conn.cursor()

In [95]:
c.execute('CREATE TABLE "History" ("Date" INTEGER, \
          "coin" TEXT, \
          "Open" FLOAT,\
          "High" FLOAT,\
          "Low" FLOAT,\
          "Close" FLOAT,\
          "Volume" FLOAT,\
          PRIMARY KEY("Date","coin"))')
conn.commit()

In [89]:
import numpy as np

In [96]:
ticker=np.array(pd.read_csv('./Raw Data/Ticker.csv',usecols=[0]).values,dtype=np.str)
ticker=np.squeeze(ticker)
ticker

array(['MSFT', 'AAPL', 'AMZN', 'FB', 'BRKB', 'JPM', 'GOOG', 'GOOGL',
       'JNJ', 'V', 'PG'], dtype='<U5')

In [97]:
for coin in ticker:
    get_stock_table(coin).to_sql('History', conn, if_exists='append', index = False)